# TP: Generalized Assignment Problem

A company wants to perform $n$ tasks using $m$ machines with minimum cost. Let the set of machines be given by $M=\{1,\ldots,m\}$ and the set of tasks be given by $N=\{1,\ldots,n\}$. We denote the cost of performing task $i$ on machine $j$ by $p_{ij}$ and the time it takes to perform task $i$ on machine $j$ by $t_{ij}$ for $i\in N$ and $j\in M$. The total capacity of each machine is $t^{max}_{j}$ for $j\in M$. The problem of finding a minimum cost assignment of tasks to machines that respects machine capacities and performs all tasks is called the generalized assignment problem. We will refer to it as GAP in the following.

The goal of this TP is to implement the branch-and-price algorithm for GAP and compare it to the direct solution of its mixed integer programming formulation. This implementation will be done using the Julia language 
with the help of several packages such as JuMP which includes the mathematical programming language macros and Gurobi which interfaces with the Gurobi solver.  

In [28]:
import Pkg
using JuMP
using Cbc
using DelimitedFiles
using NBInclude

const ϵ = 0.00001

1.0e-5

In [2]:
using JuMP
#import Pkg
#using Gurobi
using Cbc
#Pkg.add("NBInclude")
using DelimitedFiles
using NBInclude
#const GUROBI_ENV = Gurobi.Env()
const ϵ = 0.00001

   Updating registry at `C:\Users\xgodw\.julia\registries\General`
┌ Error: mktempdir cleanup
│   exception = (InterruptException(), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000000962ba15, Ptr{Nothing} @0x0000000009d2454b, Ptr{Nothing} @0x00000000097d74c8, Ptr{Nothing} @0x00000000097d7c3c, Ptr{Nothing} @0x000000000200683c, Ptr{Nothing} @0x0000000018551b39, Ptr{Nothing} @0x00000000185520f8, Ptr{Nothing} @0x0000000018550b2f, Ptr{Nothing} @0x0000000018546ef1, Ptr{Nothing} @0x000000001854c649, Ptr{Nothing} @0x000000001854cdde, Ptr{Nothing} @0x00000000184dcd00, Ptr{Nothing} @0x00000000184e03b8, Ptr{Nothing} @0x00000000184e043d, Ptr{Nothing} @0x000000006900a65c, Ptr{Nothing} @0x000000006900a321, Ptr{Nothing} @0x000000006900ae30, Ptr{Nothing} @0x000000006900b805, Base.InterpreterIP in top-level CodeInfo for Main at statement 1, Ptr{Nothing} @0x0000000069028efd, Ptr{Nothing} @0x0000000068ffd65b, Ptr{Nothing} @0x00000000099f45ef, Ptr{Nothing} @0x0000000001facfaf, Ptr{Nothing} 

1.0e-5

The TP is structured in such a way that different parts of the code are hosted in different files. Below we export them so that their functions are callable from the main function.

In [80]:
@nbinclude("typedef.ipynb")
@nbinclude("data.ipynb")
@nbinclude("MIP.ipynb")
#@nbinclude("master.ipynb")
#@nbinclude("subproblem.ipynb")
#@nbinclude("branch_and_price.ipynb")
#@nbinclude("node.ipynb")

solve_MIP (generic function with 1 method)

## Exercise 1
Open the instance file Example.txt under the folder \data, and read the files data.ipynb and typedef.ipynb under the folder \src. The file typedef.ipynb contains the definitions for two structures that will be used throughout the code. These are Data and TreeNode. Focusing on Data for the moment, familiarize yourself with how the data is read and stored in the code. 

In [81]:
global D = read_data("Example.txt")
println("Number of machines is: $(D.m)")
println("Number of jobs is: $(D.n)")
println("Machine capacities are: $(D.Tmax). This is a 1xm array.")
println("Assignment costs are: $(D.c). This is an mxn array.")
println("Processing times are: $(D.t). This is an mxn array.")

********** Read instance Example.txt ****************
Number of machines is: 3
Number of jobs is: 5
Machine capacities are: [28, 26, 18]. This is a 1xm array.
Assignment costs are: [9 19 4 29 9; 1 24 14 4 24; 24 1 34 4 14]. This is an mxn array.
Processing times are: [11 7 22 9 15; 12 16 20 10 10; 9 20 7 23 5]. This is an mxn array.


## Exercise 2
Implement and solve an integer programming model for GAP:
\begin{align*}
\min\;\;\; &\sum_{i\in N}\sum_{j\in M} p_{ij} x_{ij}\\
\mbox{s.t.}\;\;\; 	&\sum_{j\in M} x_{ij}=1 &\forall i\in N\\
&\sum_{i\in N} t_{ij}x_{ij} \leq t^{\rm max}_{j} &\forall j\in M\\
&x\in \{0,1\}^{N\times M}.
\end{align*}

To do so, open the file MIP.ipynb and complete the function solve_MIP. Once it is complete, you can test your function by calling it (don't forget to activate @nbinclude("MIP.ipynb") above).    

In [82]:
result = @timed solve_MIP()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 26.6 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 11 columns (11 integer (11 of which binary)) and 27 elements
Cbc0012I Integer solution of 85 found by DiveCoefficient after 0 iterations and 0 nodes (0.12 seconds)
Cbc0038I Full problem 6 rows 11 columns, reduced to 1 rows 2 columns
Cbc0012I Integer solution of 72 found by RINS after 0 iterations and 0 nodes (0.19 seconds)
Cbc0012I Integer solution of 67 found by DiveCoefficient after 4 iterations and 0 nodes (0.23 seconds)
Cbc0031I 3 added rows had average density of 

(value = (67.0, [0.0 1.0 … 0.0 1.0; 1.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 0.0]), time = 0.28372, bytes = 4088475, gctime = 0.0, gcstats = Base.GC_Diff(4088475, 0, 0, 75753, 17, 0, 0, 0, 0))

In [79]:
result.value[2]

3×5 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0

## Exercise 3

We will now begin implementing the branch-and-price algorithm for GAP. To do so, we first need to implement the column generation algorithm for the root node of the branching tree. 

- Open first branch_and_price.ipynb. The function solve_BP() will pilot the branch_and_price algorithm.
- Familiarize yourself with the data structure used to store the columns.
- Note that an artificial pattern as well as an empty pattern is added to the column pool of each machine.
- A list of nodes to process is created and initialized with the root node.
- We will initially focus on the function Process_Node() that will perform the column generation algorithm for each node. 



1. Open node.ipynb and master.ipynb. Complete the function node_master(), ignore the function calculate_columns() for the moment. This function builds the restricted master LP given as:
\begin{align*}
\min\;\;\; &\sum_{j\in M}\sum_{k\in K_{j}} \bar{p}^{k}_{j} \alpha^{k}_{j}\\
\mbox{s.t.}\;\;\; 	&\sum_{j\in M}\sum_{k\in K_{j}} \bar{x}^{k}_{ij}\alpha^{k}_{j}=1 &\forall i\in N\\
&\sum_{k\in K_{j}} \alpha^{k}_{j}=1&\forall j\in M\\
&\alpha_{j}\in [0,1]^{|K_{j}|}&\forall j\in M.
\end{align*}
where $K_{j}$ represents the initial set of columns for machine $j$, $\bar{p}^{k}_{j}$ is the cost of column $k$ on machine $j$, and $\bar{x}$ is a vector of assignment of tasks to machine $j$ (a pattern vector).

2. Open subproblem.ipynb. Complete the function machine_subproblem() with the appropriate objective function. We will not be concerned with the branching constraints for the moment.

3. Open node.ipynb. Complete the nodeub, nodelb and reduced cost values in the function Process_Node(). 

4. Open node.ipynb. Complete the part # ADD COLUMN TO MODEL in the function Process_Node().

5. Open node.ipynb. Complete the function calculate_xsol().

You may now test your column generation algorithm at the root node by calling the function solve_BP() (don't forget to activate @nbinclude(branch_and_price.ipynb),@nbinclude("node.ipynb"),@nbinclude("master.ipynb"),@nbinclude("subproblem.ipynb") above).

In [ ]:
result= @timed solve_BP()

## Exercise 4

We will now focus on the necessary modifications to extend the column generation algorithm applied at the root node to any node of the branch-and-bound tree. To do so:

1. Open branch_and_price.ipynb. Complete the function calculate_branching().

2. Open node.ipynb. Complete the function calculate_columns() to filter out the columns that are consistent with the branching constraints of each node.

3. Open subproblem.ipynb. Revisit the function machine_subproblem() to incorporate the branching constraints.

4. Read the entirety of the function solve_BP(). Pay special attention to how new nodes are added to the tree.

You may now test your branch_and_price algorithm by calling the function solve_BP().

In [ ]:
result= @timed solve_BP()